In [2]:
import numpy as np
import pandas as pd
import glob

## Datos de Comercio

* En el paper original se usan productos desagregados según la SITC revisión 4 a nivel de cuatro dígitos, de la base de datos de UNCOMTRADE. 

* En este caso se utilizarán datos de WITS provenientes de UNComtrade correspondientes al sistema armonizado (HS) a 4 dígitos para los años 1997 a 1999, 2007 a 2009 y 2017 a 2019. 

* La idea es calcular los grafos/redes para 3 momentos del tiempo distintos: comparar el comportamiento promedio (para cada grupo de 3 años) de cada país en los distintos momentos del tiempo. 

### Descargo datos de WITS y leo los archivos

* Se descargaron por partes los datos porque la consulta no puede exceder de 100 mil filas. 

* Fecha de descarga: 15 de enero 2021. http://wits.worldbank.org/

* Año por año se descargaron las exportaciones al Mundo a 4 dígitos del SA en dos grupos de países: con nombres de A a L (A_L) y M a Z (M_Z).

* Detalles de la consulta (query name: 4digproductspace) => ALL2 -- Heading (all 4-digit HS codes) HS 1996 (Selected Classification),  Trade Flow Exports y Partners World -- WLD

In [3]:
# leo todos los archivos descargados y los agrupo en un mismo dataframe
# fuente del código: <https://stackoverflow.com/questions/20906474/import-multiple-csv-files-into-pandas-and-concatenate-into-one-dataframe>
path = r'./'
# tomo el nombre de todos los archivos con terminación .csv
all_files = glob.glob(path + "/*.csv")
# genero un loop para leer cada elemento de la lista de archivos
li = []
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    # agrego a la lista todos los dataframe individuales
    li.append(df)
# concateno todos los df de la lista
df = pd.concat(li, axis=0, ignore_index=True)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1018163 entries, 0 to 1018162
Data columns (total 10 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   Nomenclature            1018163 non-null  object 
 1   ReporterISO3            1018163 non-null  object 
 2   ProductCode             1018163 non-null  int64  
 3   ReporterName            1018163 non-null  object 
 4   PartnerISO3             1018163 non-null  object 
 5   PartnerName             1018163 non-null  object 
 6   Year                    1018163 non-null  int64  
 7   TradeFlowName           1018163 non-null  object 
 8   TradeFlowCode           1018163 non-null  int64  
 9   TradeValue in 1000 USD  1018163 non-null  float64
dtypes: float64(1), int64(3), object(6)
memory usage: 77.7+ MB


In [5]:
df

,Nomenclature,ReporterISO3,ProductCode,ReporterName,PartnerISO3,PartnerName,Year,TradeFlowName,TradeFlowCode,TradeValue in 1000 USD
0,H1,ALB,101,Albania,WLD,World,1997,Export,6,14.538
1,H1,ALB,105,Albania,WLD,World,1997,Export,6,10.612
2,H1,ALB,106,Albania,WLD,World,1997,Export,6,34.767
3,H1,ALB,206,Albania,WLD,World,1997,Export,6,18.225
4,H1,ALB,207,Albania,WLD,World,1997,Export,6,105.227
...,...,...,...,...,...,...,...,...,...,...
1018158,H1,ZWE,9618,Zimbabwe,WLD,World,2019,Export,6,0.456
1018159,H1,ZWE,9701,Zimbabwe,WLD,World,2019,Export,6,32.100
1018160,H1,ZWE,9703,Zimbabwe,WLD,World,2019,Export,6,2172.791
1018161,H1,ZWE,9705,Zimbabwe,WLD,World,2019,Export,6,8252.694


In [31]:
df.groupby(by = 'Year').count()

,Nomenclature,ReporterISO3,ProductCode,ReporterName,PartnerISO3,PartnerName,TradeFlowName,TradeFlowCode,TradeValue in 1000 USD
Year,,,,,,,,,
1997,72901,72901,72901,72901,72901,72901,72901,72901,72901
1998,84720,84720,84720,84720,84720,84720,84720,84720,84720
1999,96132,96132,96132,96132,96132,96132,96132,96132,96132
2007,128697,128697,128697,128697,128697,128697,128697,128697,128697
2008,129378,129378,129378,129378,129378,129378,129378,129378,129378
2009,132133,132133,132133,132133,132133,132133,132133,132133,132133
2017,133220,133220,133220,133220,133220,133220,133220,133220,133220
2018,127526,127526,127526,127526,127526,127526,127526,127526,127526
2019,113456,113456,113456,113456,113456,113456,113456,113456,113456


In [32]:
df.groupby(by = ['ReporterISO3', 'Year']).count()

Nomenclature  ProductCode  ReporterName  PartnerISO3  \
ReporterISO3 Year                                                         
ABW          2007           212          212           212          212   
             2008           205          205           205          205   
             2009           540          540           540          540   
             2017           516          516           516          516   
             2018           511          511           511          511   
...                         ...          ...           ...          ...   
ZWE          2008           702          702           702          702   
             2009           673          673           673          673   
             2017           645          645           645          645   
             2018           643          643           643          643   
             2019           637          637           637          637   

                   PartnerName  TradeFlowName  TradeFlowCode  \
ReporterISO3 Year                                              
ABW          2007          212            212            212   
             2008          205            205            205   
             2009          540            540            540   
             2017          516            516            516   
             2018          511            511            511   
...                        ...            ...            ...   
ZWE          2008          702            702            702   
             2009          673            673            673   
             2017          645            645            645   
             2018          643            643            643   
             2019          637            637            637   

                   TradeValue in 1000 USD  
ReporterISO3 Year                          
ABW          2007                     212  
             2008                     205  
             2009                     540  
             2017                     516  
             2018                     511  
...                                   ...  
ZWE          2008                     702  
             2009                     673  
             2017                     645  
             2018                     643  
             2019                     637  

[1270 rows x 8 columns]

In [55]:
# chequeo los valores unicos por columna de interés
print(df.Year.unique()) # 9 años en distintos períodos (últimos 20 años)
print(len(df.ProductCode.unique())) # 1224 productos a 4 dígitos del SA
print(df.ProductCode.unique())
print(len(df.PartnerISO3.unique())) # un solo partner = world
print(df.PartnerISO3.unique())
print(len(df.ReporterISO3.unique())) # 193 reporters 
print(df.ReporterISO3.unique())

[1997 1998 1999 2007 2008 2009 2017 2018 2019]
1242
[ 101  105  106 ... 5005 7401 2613]
1
['WLD']
193
['ALB' 'AND' 'ARG' 'ARM' 'AUS' 'AUT' 'BGR' 'BHS' 'BLX' 'BOL' 'BRA' 'CAF'
 'CAN' 'CHL' 'CHN' 'COL' 'CPV' 'CRI' 'CYP' 'DEU' 'DZA' 'EST' 'FIN' 'FRA'
 'FRO' 'GAB' 'GMB' 'GRC' 'GRL' 'GTM' 'GUY' 'HKG' 'HND' 'HRV' 'HTI' 'HUN'
 'IDN' 'IND' 'IRL' 'IRN' 'ISL' 'ISR' 'ITA' 'JPN' 'KEN' 'KOR' 'LBN' 'LTU'
 'LVA' 'PYF' 'CHE' 'ESP' 'GBR' 'MAC' 'MDG' 'MDV' 'MEX' 'MKD' 'MLT' 'MNG'
 'MUS' 'MYS' 'NIC' 'NLD' 'NOR' 'NZL' 'POL' 'PRT' 'ROM' 'RUS' 'SEN' 'SGP'
 'SVK' 'SVN' 'SWE' 'SYC' 'TUR' 'TZA' 'UGA' 'URY' 'USA' 'VEN' 'ZAF' 'ZMB'
 'ANT' 'ARE' 'KNA' 'LKA' 'MLI' 'MOZ' 'MSR' 'MWI' 'NER' 'NGA' 'NPL' 'PAN'
 'PER' 'PRY' 'SAU' 'STP' 'TCA' 'TGO' 'THA' 'TKM' 'TTO' 'VCT' 'BEN' 'BLR'
 'BLZ' 'CZE' 'DNK' 'ECU' 'GEO' 'GIN' 'JAM' 'JOR' 'KAZ' 'SLV' 'ATG' 'AZE'
 'BEL' 'BTN' 'CUB' 'DMA' 'LUX' 'SDN' 'ABW' 'AGO' 'BDI' 'BFA' 'BGD' 'BHR'
 'BIH' 'BRB' 'BRN' 'BWA' 'CIV' 'CMR' 'COG' 'DOM' 'ETH' 'EUN' 'FJI' 'GHA'
 'GRD' 'KGZ' 'KHM' 'KI

In [61]:
# chequeo NAs
print(df.Year.isna().sum()) 
print(df.ProductCode.isna().sum()) 
print(df.PartnerISO3.isna().sum())
print(df.ReporterISO3.isna().sum())
print(df[['TradeValue in 1000 USD']].isna().sum())

0
0
0
0
TradeValue in 1000 USD    0
dtype: int64


In [82]:
df_wide = df[["ReporterISO3", "ProductCode", 'ReporterName', 'Year', 'TradeValue in 1000 USD']]

In [83]:
df_wide = df_wide.pivot(index=["ReporterISO3", "ProductCode", 'ReporterName'], columns='Year', values='TradeValue in 1000 USD')
df_wide

Year                                   1997  1998  1999       2007       2008  \
ReporterISO3 ProductCode ReporterName                                           
ABW          101         Aruba          NaN   NaN   NaN      1.809      1.285   
             103         Aruba          NaN   NaN   NaN        NaN        NaN   
             104         Aruba          NaN   NaN   NaN      0.642      0.017   
             105         Aruba          NaN   NaN   NaN      6.849      1.531   
             106         Aruba          NaN   NaN   NaN      5.834     10.069   
...                                     ...   ...   ...        ...        ...   
ZWE          9702        Zimbabwe       NaN   NaN   NaN        NaN      0.002   
             9703        Zimbabwe       NaN   NaN   NaN   4844.384    690.542   
             9704        Zimbabwe       NaN   NaN   NaN      2.376        NaN   
             9705        Zimbabwe       NaN   NaN   NaN  30099.052  29621.528   
             9999        Zimbabwe       NaN   NaN   NaN   9485.615        NaN   

Year                                        2009      2017      2018      2019  
ReporterISO3 ProductCode ReporterName                                           
ABW          101         Aruba             4.476     6.145     5.397     8.483  
             103         Aruba             0.084       NaN       NaN       NaN  
             104         Aruba               NaN       NaN       NaN       NaN  
             105         Aruba             4.473    27.374    73.884    47.781  
             106         Aruba            10.007    12.024    10.594     4.669  
...                                          ...       ...       ...       ...  
ZWE          9702        Zimbabwe            NaN     0.655       NaN       NaN  
             9703        Zimbabwe        966.129  2261.126  2253.457  2172.791  
             9704        Zimbabwe            NaN       NaN       NaN       NaN  
             9705        Zimbabwe      14193.067  8866.771  8995.939  8252.694  
             9999        Zimbabwe         17.901   398.377   500.855  1450.204  

[179602 rows x 9 columns]

In [84]:
# creo columnas promedio para los años de interés
col = df_wide.loc[: , '2017':'2019']
col2 = df_wide.loc[: , '2007':'2009']
col3 = df_wide.loc[: , '1997':'1999']
df_wide['prom_1719'] = round(col.mean(axis=1),2)
df_wide['prom_0709'] = round(col2.mean(axis=1),2)
df_wide['prom_9799'] = round(col3.mean(axis=1),2)

Al calcular el promedio aplicando mean de pandas automáticamente se excluyen los NaN values. El parámetro por default de mean es skipna = True. Si quisieramos que no los excluya deberíamos setearlo en False. En este caso nos interesa el promedio de los 3 años considerados, sin importar si alguno de los años no se detectó comercio. Por tal motivo dejamos la función mean como viene en default. 

In [85]:
df_wide

Year                                   1997  1998  1999       2007       2008  \
ReporterISO3 ProductCode ReporterName                                           
ABW          101         Aruba          NaN   NaN   NaN      1.809      1.285   
             103         Aruba          NaN   NaN   NaN        NaN        NaN   
             104         Aruba          NaN   NaN   NaN      0.642      0.017   
             105         Aruba          NaN   NaN   NaN      6.849      1.531   
             106         Aruba          NaN   NaN   NaN      5.834     10.069   
...                                     ...   ...   ...        ...        ...   
ZWE          9702        Zimbabwe       NaN   NaN   NaN        NaN      0.002   
             9703        Zimbabwe       NaN   NaN   NaN   4844.384    690.542   
             9704        Zimbabwe       NaN   NaN   NaN      2.376        NaN   
             9705        Zimbabwe       NaN   NaN   NaN  30099.052  29621.528   
             9999        Zimbabwe       NaN   NaN   NaN   9485.615        NaN   

Year                                        2009      2017      2018  \
ReporterISO3 ProductCode ReporterName                                  
ABW          101         Aruba             4.476     6.145     5.397   
             103         Aruba             0.084       NaN       NaN   
             104         Aruba               NaN       NaN       NaN   
             105         Aruba             4.473    27.374    73.884   
             106         Aruba            10.007    12.024    10.594   
...                                          ...       ...       ...   
ZWE          9702        Zimbabwe            NaN     0.655       NaN   
             9703        Zimbabwe        966.129  2261.126  2253.457   
             9704        Zimbabwe            NaN       NaN       NaN   
             9705        Zimbabwe      14193.067  8866.771  8995.939   
             9999        Zimbabwe         17.901   398.377   500.855   

Year                                       2019  prom_1719  prom_0709  \
ReporterISO3 ProductCode ReporterName                                   
ABW          101         Aruba            8.483       6.68       2.52   
             103         Aruba              NaN        NaN       0.08   
             104         Aruba              NaN        NaN       0.33   
             105         Aruba           47.781      49.68       4.28   
             106         Aruba            4.669       9.10       8.64   
...                                         ...        ...        ...   
ZWE          9702        Zimbabwe           NaN       0.66       0.00   
             9703        Zimbabwe      2172.791    2229.12    2167.02   
             9704        Zimbabwe           NaN        NaN       2.38   
             9705        Zimbabwe      8252.694    8705.13   24637.88   
             9999        Zimbabwe      1450.204     783.15    4751.76   

Year                                   prom_9799  
ReporterISO3 ProductCode ReporterName             
ABW          101         Aruba               NaN  
             103         Aruba               NaN  
             104         Aruba               NaN  
             105         Aruba               NaN  
             106         Aruba               NaN  
...                                          ...  
ZWE          9702        Zimbabwe            NaN  
             9703        Zimbabwe            NaN  
             9704        Zimbabwe            NaN  
             9705        Zimbabwe            NaN  
             9999        Zimbabwe            NaN  

[179602 rows x 12 columns]

In [86]:
type(df_wide)

pandas.core.frame.DataFrame

In [87]:
df_wide.describe()

Year,1997,1998,1999,2007,2008,2009,2017,2018,2019,prom_1719,prom_0709,prom_9799
count,7.290100e+04,8.472000e+04,9.613200e+04,1.286970e+05,1.293780e+05,1.321330e+05,1.332200e+05,1.275260e+05,1.134560e+05,1.472830e+05,1.521100e+05,1.081840e+05
mean,6.431616e+04,5.587942e+04,5.215971e+04,1.147688e+05,1.318818e+05,1.005682e+05,1.430022e+05,1.630614e+05,1.772607e+05,1.368472e+05,9.935662e+04,4.758459e+04
std,5.651620e+05,5.294624e+05,5.528767e+05,1.409623e+06,1.711068e+06,1.246613e+06,1.779057e+06,2.047784e+06,2.103057e+06,1.808991e+06,1.333812e+06,5.009592e+05
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,3.439900e+01,2.260825e+01,1.127875e+01,9.803000e+00,1.219700e+01,9.284000e+00,1.436775e+01,1.699700e+01,2.457350e+01,1.115000e+01,6.220000e+00,9.440000e+00
50%,7.636380e+02,5.378010e+02,3.131445e+02,3.491050e+02,4.198780e+02,3.255860e+02,4.952550e+02,5.950335e+02,8.483070e+02,3.271800e+02,1.834950e+02,2.319800e+02
75%,1.115399e+04,8.617105e+03,6.281695e+03,9.033279e+03,1.045439e+04,8.037102e+03,1.181714e+04,1.414293e+04,1.849791e+04,9.121695e+03,5.751085e+03,5.285400e+03
max,5.297055e+07,6.035613e+07,6.224192e+07,1.783808e+08,2.470972e+08,1.421942e+08,1.750316e+08,2.315906e+08,2.023822e+08,2.014225e+08,1.892241e+08,5.852287e+07


In [88]:
print(df_wide.prom_1719.isna().sum()) # 32319 nas y 147283 filas total
print(df_wide.prom_1719.count())

print(df_wide.prom_0709.isna().sum())
print(df_wide.prom_0709.count())

print(df_wide.prom_9799.isna().sum())
print(df_wide.prom_9799.count())

32319
147283
27492
152110
71418
108184


In [89]:
df_wide.prom_1719.count()

147283

In [90]:
df_wide.sum(numeric_only=True)

Year
1997         4.688713e+09
1998         4.734105e+09
1999         5.014217e+09
2007         1.477040e+10
2008         1.706260e+10
2009         1.328838e+10
2017         1.905076e+10
2018         2.079456e+10
2019         2.011129e+10
prom_1719    2.015527e+10
prom_0709    1.511313e+10
prom_9799    5.147891e+09
dtype: float64

In [91]:
df_wide.index

MultiIndex([('ABW',  101,    'Aruba'),
            ('ABW',  103,    'Aruba'),
            ('ABW',  104,    'Aruba'),
            ('ABW',  105,    'Aruba'),
            ('ABW',  106,    'Aruba'),
            ('ABW',  201,    'Aruba'),
            ('ABW',  202,    'Aruba'),
            ('ABW',  203,    'Aruba'),
            ('ABW',  204,    'Aruba'),
            ('ABW',  206,    'Aruba'),
            ...
            ('ZWE', 9615, 'Zimbabwe'),
            ('ZWE', 9616, 'Zimbabwe'),
            ('ZWE', 9617, 'Zimbabwe'),
            ('ZWE', 9618, 'Zimbabwe'),
            ('ZWE', 9701, 'Zimbabwe'),
            ('ZWE', 9702, 'Zimbabwe'),
            ('ZWE', 9703, 'Zimbabwe'),
            ('ZWE', 9704, 'Zimbabwe'),
            ('ZWE', 9705, 'Zimbabwe'),
            ('ZWE', 9999, 'Zimbabwe')],
           names=['ReporterISO3', 'ProductCode', 'ReporterName'], length=179602)

In [92]:
# armo un data frame con los datos por producto de todos los países que será el total mundial con el que trabajaré
WLD_tot = df_wide.groupby(["ProductCode"]).sum()

In [93]:
WLD_tot.head()

Year,1997,1998,1999,2007,2008,2009,2017,2018,2019,prom_1719,prom_0709,prom_9799
ProductCode,,,,,,,,,,,,
101,1149995.020,1142210.785,1311227.527,3021137.476,2986103.832,2583116.125,3403169.662,3.661081e+06,3.927005e+06,3667819.69,2864787.07,1223045.90
102,4211472.352,4354119.554,4087258.844,6757029.938,7127564.900,6778566.292,9879076.088,1.095508e+07,1.022768e+07,10483561.30,6916207.50,4470777.67
103,1346754.711,1371320.018,1300971.310,3325553.021,4374910.008,4421164.389,4617310.833,4.024561e+06,4.490071e+06,4380636.22,4042493.17,1507011.69
104,590534.329,512192.274,495892.964,979864.966,1208863.649,1232765.388,1916448.139,1.825421e+06,1.466535e+06,1970374.07,1206758.60,587566.95
105,912940.761,921171.499,890194.365,2008192.191,2458323.950,2579648.479,3316300.716,3.652797e+06,3.620554e+06,3539346.98,2349411.99,955992.76


In [94]:
WLD_tot.loc['Total'] = WLD_tot.sum(numeric_only=True)

In [96]:
WLD_tot

Year,1997,1998,1999,2007,2008,2009,2017,2018,2019,prom_1719,prom_0709,prom_9799
ProductCode,,,,,,,,,,,,
101,1.149995e+06,1.142211e+06,1.311228e+06,3.021137e+06,2.986104e+06,2.583116e+06,3.403170e+06,3.661081e+06,3.927005e+06,3.667820e+06,2.864787e+06,1.223046e+06
102,4.211472e+06,4.354120e+06,4.087259e+06,6.757030e+06,7.127565e+06,6.778566e+06,9.879076e+06,1.095508e+07,1.022768e+07,1.048356e+07,6.916208e+06,4.470778e+06
103,1.346755e+06,1.371320e+06,1.300971e+06,3.325553e+06,4.374910e+06,4.421164e+06,4.617311e+06,4.024561e+06,4.490071e+06,4.380636e+06,4.042493e+06,1.507012e+06
104,5.905343e+05,5.121923e+05,4.958930e+05,9.798650e+05,1.208864e+06,1.232765e+06,1.916448e+06,1.825421e+06,1.466535e+06,1.970374e+06,1.206759e+06,5.875669e+05
105,9.129408e+05,9.211715e+05,8.901944e+05,2.008192e+06,2.458324e+06,2.579648e+06,3.316301e+06,3.652797e+06,3.620554e+06,3.539347e+06,2.349412e+06,9.559928e+05
...,...,...,...,...,...,...,...,...,...,...,...,...
9704,2.039868e+05,1.748930e+05,1.619459e+05,2.360655e+05,2.376180e+05,2.069267e+05,1.719148e+05,1.864708e+05,1.600196e+05,1.735062e+05,2.281169e+05,1.852215e+05
9705,2.854066e+05,3.425063e+05,3.369923e+05,7.005600e+05,9.198514e+05,6.852953e+05,1.596656e+06,1.682814e+06,2.098486e+06,1.831003e+06,7.690941e+05,3.399055e+05
9706,1.445352e+06,1.337021e+06,1.485024e+06,3.266286e+06,3.093562e+06,1.908123e+06,3.154910e+06,3.511998e+06,4.206318e+06,3.651594e+06,2.788744e+06,1.468292e+06


In [104]:
df_wide.sum(numeric_only=True) == WLD_tot.loc['Total'] # no dan exacto igual pero quizás es por los decimales

Year
1997         False
1998         False
1999         False
2007          True
2008          True
2009         False
2017          True
2018          True
2019         False
prom_1719    False
prom_0709    False
prom_9799    False
dtype: bool

In [109]:
WLD_tot.loc['Total']

Year
1997         4.688713e+09
1998         4.734105e+09
1999         5.014217e+09
2007         1.477040e+10
2008         1.706260e+10
2009         1.328838e+10
2017         1.905076e+10
2018         2.079456e+10
2019         2.011129e+10
prom_1719    2.015527e+10
prom_0709    1.511313e+10
prom_9799    5.147891e+09
Name: Total, dtype: float64

In [112]:
df_wide.sum(numeric_only=True)

Year
1997         4.688713e+09
1998         4.734105e+09
1999         5.014217e+09
2007         1.477040e+10
2008         1.706260e+10
2009         1.328838e+10
2017         1.905076e+10
2018         2.079456e+10
2019         2.011129e+10
prom_1719    2.015527e+10
prom_0709    1.511313e+10
prom_9799    5.147891e+09
dtype: float64

In [115]:
# agrego columnas de todos los países
WLD_tot['ReporterISO3'] = np.repeat('Todos los paises',len(WLD_tot),axis=0)
WLD_tot['ReporterName'] = np.repeat('TOT',len(WLD_tot),axis=0)

In [117]:
WLD_tot

Year,1997,1998,1999,2007,2008,2009,2017,2018,2019,prom_1719,prom_0709,prom_9799,ReporterISO3,ReporterName
ProductCode,,,,,,,,,,,,,,
101,1.149995e+06,1.142211e+06,1.311228e+06,3.021137e+06,2.986104e+06,2.583116e+06,3.403170e+06,3.661081e+06,3.927005e+06,3.667820e+06,2.864787e+06,1.223046e+06,Todos los paises,TOT
102,4.211472e+06,4.354120e+06,4.087259e+06,6.757030e+06,7.127565e+06,6.778566e+06,9.879076e+06,1.095508e+07,1.022768e+07,1.048356e+07,6.916208e+06,4.470778e+06,Todos los paises,TOT
103,1.346755e+06,1.371320e+06,1.300971e+06,3.325553e+06,4.374910e+06,4.421164e+06,4.617311e+06,4.024561e+06,4.490071e+06,4.380636e+06,4.042493e+06,1.507012e+06,Todos los paises,TOT
104,5.905343e+05,5.121923e+05,4.958930e+05,9.798650e+05,1.208864e+06,1.232765e+06,1.916448e+06,1.825421e+06,1.466535e+06,1.970374e+06,1.206759e+06,5.875669e+05,Todos los paises,TOT
105,9.129408e+05,9.211715e+05,8.901944e+05,2.008192e+06,2.458324e+06,2.579648e+06,3.316301e+06,3.652797e+06,3.620554e+06,3.539347e+06,2.349412e+06,9.559928e+05,Todos los paises,TOT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9704,2.039868e+05,1.748930e+05,1.619459e+05,2.360655e+05,2.376180e+05,2.069267e+05,1.719148e+05,1.864708e+05,1.600196e+05,1.735062e+05,2.281169e+05,1.852215e+05,Todos los paises,TOT
9705,2.854066e+05,3.425063e+05,3.369923e+05,7.005600e+05,9.198514e+05,6.852953e+05,1.596656e+06,1.682814e+06,2.098486e+06,1.831003e+06,7.690941e+05,3.399055e+05,Todos los paises,TOT
9706,1.445352e+06,1.337021e+06,1.485024e+06,3.266286e+06,3.093562e+06,1.908123e+06,3.154910e+06,3.511998e+06,4.206318e+06,3.651594e+06,2.788744e+06,1.468292e+06,Todos los paises,TOT


In [129]:
WLD_tot.set_index(["ReporterISO3",'ReporterName'])

Year                                   1997          1998          1999  \
ReporterISO3     ReporterName                                             
Todos los paises TOT           1.149995e+06  1.142211e+06  1.311228e+06   
                 TOT           4.211472e+06  4.354120e+06  4.087259e+06   
                 TOT           1.346755e+06  1.371320e+06  1.300971e+06   
                 TOT           5.905343e+05  5.121923e+05  4.958930e+05   
                 TOT           9.129408e+05  9.211715e+05  8.901944e+05   
...                                     ...           ...           ...   
                 TOT           2.039868e+05  1.748930e+05  1.619459e+05   
                 TOT           2.854066e+05  3.425063e+05  3.369923e+05   
                 TOT           1.445352e+06  1.337021e+06  1.485024e+06   
                 TOT           1.199337e+08  1.107682e+08  1.206252e+08   
                 TOT           4.688713e+09  4.734105e+09  5.014217e+09   

Year                                   2007          2008          2009  \
ReporterISO3     ReporterName                                             
Todos los paises TOT           3.021137e+06  2.986104e+06  2.583116e+06   
                 TOT           6.757030e+06  7.127565e+06  6.778566e+06   
                 TOT           3.325553e+06  4.374910e+06  4.421164e+06   
                 TOT           9.798650e+05  1.208864e+06  1.232765e+06   
                 TOT           2.008192e+06  2.458324e+06  2.579648e+06   
...                                     ...           ...           ...   
                 TOT           2.360655e+05  2.376180e+05  2.069267e+05   
                 TOT           7.005600e+05  9.198514e+05  6.852953e+05   
                 TOT           3.266286e+06  3.093562e+06  1.908123e+06   
                 TOT           5.884525e+08  7.235735e+08  6.607815e+08   
                 TOT           1.477040e+10  1.706260e+10  1.328838e+10   

Year                                   2017          2018          2019  \
ReporterISO3     ReporterName                                             
Todos los paises TOT           3.403170e+06  3.661081e+06  3.927005e+06   
                 TOT           9.879076e+06  1.095508e+07  1.022768e+07   
                 TOT           4.617311e+06  4.024561e+06  4.490071e+06   
                 TOT           1.916448e+06  1.825421e+06  1.466535e+06   
                 TOT           3.316301e+06  3.652797e+06  3.620554e+06   
...                                     ...           ...           ...   
                 TOT           1.719148e+05  1.864708e+05  1.600196e+05   
                 TOT           1.596656e+06  1.682814e+06  2.098486e+06   
                 TOT           3.154910e+06  3.511998e+06  4.206318e+06   
                 TOT           8.181851e+08  9.128963e+08  8.540004e+08   
                 TOT           1.905076e+10  2.079456e+10  2.011129e+10   

Year                              prom_1719     prom_0709     prom_9799  
ReporterISO3     ReporterName                                            
Todos los paises TOT           3.667820e+06  2.864787e+06  1.223046e+06  
                 TOT           1.048356e+07  6.916208e+06  4.470778e+06  
                 TOT           4.380636e+06  4.042493e+06  1.507012e+06  
                 TOT           1.970374e+06  1.206759e+06  5.875669e+05  
                 TOT           3.539347e+06  2.349412e+06  9.559928e+05  
...                                     ...           ...           ...  
                 TOT           1.735062e+05  2.281169e+05  1.852215e+05  
                 TOT           1.831003e+06  7.690941e+05  3.399055e+05  
                 TOT           3.651594e+06  2.788744e+06  1.468292e+06  
                 TOT           8.623582e+08  6.588180e+08  1.220377e+08  
                 TOT           2.015527e+10  1.511313e+10  5.147891e+09  

[1243 rows x 12 columns]

In [130]:
df_wide.reset_index()

Year,ReporterISO3,ProductCode,ReporterName,1997,1998,1999,2007,2008,2009,2017,2018,2019,prom_1719,prom_0709,prom_9799
0,ABW,101,Aruba,NaN,NaN,NaN,1.809,1.285,4.476,6.145,5.397,8.483,6.68,2.52,NaN
1,ABW,103,Aruba,NaN,NaN,NaN,NaN,NaN,0.084,NaN,NaN,NaN,NaN,0.08,NaN
2,ABW,104,Aruba,NaN,NaN,NaN,0.642,0.017,NaN,NaN,NaN,NaN,NaN,0.33,NaN
3,ABW,105,Aruba,NaN,NaN,NaN,6.849,1.531,4.473,27.374,73.884,47.781,49.68,4.28,NaN
4,ABW,106,Aruba,NaN,NaN,NaN,5.834,10.069,10.007,12.024,10.594,4.669,9.10,8.64,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179597,ZWE,9702,Zimbabwe,NaN,NaN,NaN,NaN,0.002,NaN,0.655,NaN,NaN,0.66,0.00,NaN
179598,ZWE,9703,Zimbabwe,NaN,NaN,NaN,4844.384,690.542,966.129,2261.126,2253.457,2172.791,2229.12,2167.02,NaN
179599,ZWE,9704,Zimbabwe,NaN,NaN,NaN,2.376,NaN,NaN,NaN,NaN,NaN,NaN,2.38,NaN
179600,ZWE,9705,Zimbabwe,NaN,NaN,NaN,30099.052,29621.528,14193.067,8866.771,8995.939,8252.694,8705.13,24637.88,NaN


In [126]:
WLD_tot.index.names

FrozenList(['ProductCode'])

In [131]:
# reseteo los index para unir luego las tablas 
df_w = df_wide.reset_index()
df_tot = WLD_tot.reset_index()
# uno tablas y creao nuevo index con las columnas correspondientes 
df_tot = df_w.append(df_tot).set_index(["ReporterISO3", "ProductCode", 'ReporterName'])

In [133]:
df_tot

1997          1998  \
ReporterISO3     ProductCode ReporterName                               
ABW              101         Aruba                  NaN           NaN   
                 103         Aruba                  NaN           NaN   
                 104         Aruba                  NaN           NaN   
                 105         Aruba                  NaN           NaN   
                 106         Aruba                  NaN           NaN   
...                                                 ...           ...   
Todos los paises 9704        TOT           2.039868e+05  1.748930e+05   
                 9705        TOT           2.854066e+05  3.425063e+05   
                 9706        TOT           1.445352e+06  1.337021e+06   
                 9999        TOT           1.199337e+08  1.107682e+08   
                 Total       TOT           4.688713e+09  4.734105e+09   

                                                   1999          2007  \
ReporterISO3     ProductCode ReporterName                               
ABW              101         Aruba                  NaN  1.809000e+00   
                 103         Aruba                  NaN           NaN   
                 104         Aruba                  NaN  6.420000e-01   
                 105         Aruba                  NaN  6.849000e+00   
                 106         Aruba                  NaN  5.834000e+00   
...                                                 ...           ...   
Todos los paises 9704        TOT           1.619459e+05  2.360655e+05   
                 9705        TOT           3.369923e+05  7.005600e+05   
                 9706        TOT           1.485024e+06  3.266286e+06   
                 9999        TOT           1.206252e+08  5.884525e+08   
                 Total       TOT           5.014217e+09  1.477040e+10   

                                                   2008          2009  \
ReporterISO3     ProductCode ReporterName                               
ABW              101         Aruba         1.285000e+00  4.476000e+00   
                 103         Aruba                  NaN  8.400000e-02   
                 104         Aruba         1.700000e-02           NaN   
                 105         Aruba         1.531000e+00  4.473000e+00   
                 106         Aruba         1.006900e+01  1.000700e+01   
...                                                 ...           ...   
Todos los paises 9704        TOT           2.376180e+05  2.069267e+05   
                 9705        TOT           9.198514e+05  6.852953e+05   
                 9706        TOT           3.093562e+06  1.908123e+06   
                 9999        TOT           7.235735e+08  6.607815e+08   
                 Total       TOT           1.706260e+10  1.328838e+10   

                                                   2017          2018  \
ReporterISO3     ProductCode ReporterName                               
ABW              101         Aruba         6.145000e+00  5.397000e+00   
                 103         Aruba                  NaN           NaN   
                 104         Aruba                  NaN           NaN   
                 105         Aruba         2.737400e+01  7.388400e+01   
                 106         Aruba         1.202400e+01  1.059400e+01   
...                                                 ...           ...   
Todos los paises 9704        TOT           1.719148e+05  1.864708e+05   
                 9705        TOT           1.596656e+06  1.682814e+06   
                 9706        TOT           3.154910e+06  3.511998e+06   
                 9999        TOT           8.181851e+08  9.128963e+08   
                 Total       TOT           1.905076e+10  2.079456e+10   

                                                   2019     prom_1719  \
ReporterISO3     ProductCode ReporterName                               
ABW              101         Aruba         8.483000e+00  6.680000e+00   
                 103         Aru

In [138]:
df_tot.index

MultiIndex([(             'ABW',     101, 'Aruba'),
            (             'ABW',     103, 'Aruba'),
            (             'ABW',     104, 'Aruba'),
            (             'ABW',     105, 'Aruba'),
            (             'ABW',     106, 'Aruba'),
            (             'ABW',     201, 'Aruba'),
            (             'ABW',     202, 'Aruba'),
            (             'ABW',     203, 'Aruba'),
            (             'ABW',     204, 'Aruba'),
            (             'ABW',     206, 'Aruba'),
            ...
            ('Todos los paises',    9617,   'TOT'),
            ('Todos los paises',    9618,   'TOT'),
            ('Todos los paises',    9701,   'TOT'),
            ('Todos los paises',    9702,   'TOT'),
            ('Todos los paises',    9703,   'TOT'),
            ('Todos los paises',    9704,   'TOT'),
            ('Todos los paises',    9705,   'TOT'),
            ('Todos los paises',    9706,   'TOT'),
            ('Todos los paises',    9999,   'TOT

In [146]:
# elimino la fila que tiene el total de todos los países porque ahora voy a calcular todos los subtotales
df_tot = df_tot.drop(index=[('Todos los paises', 'Total','TOT')])
df_tot

1997          1998  \
ReporterISO3     ProductCode ReporterName                               
ABW              101         Aruba                  NaN           NaN   
                 103         Aruba                  NaN           NaN   
                 104         Aruba                  NaN           NaN   
                 105         Aruba                  NaN           NaN   
                 106         Aruba                  NaN           NaN   
...                                                 ...           ...   
Todos los paises 9703        TOT           4.995609e+05  5.288797e+05   
                 9704        TOT           2.039868e+05  1.748930e+05   
                 9705        TOT           2.854066e+05  3.425063e+05   
                 9706        TOT           1.445352e+06  1.337021e+06   
                 9999        TOT           1.199337e+08  1.107682e+08   

                                                   1999          2007  \
ReporterISO3     ProductCode ReporterName                               
ABW              101         Aruba                  NaN  1.809000e+00   
                 103         Aruba                  NaN           NaN   
                 104         Aruba                  NaN  6.420000e-01   
                 105         Aruba                  NaN  6.849000e+00   
                 106         Aruba                  NaN  5.834000e+00   
...                                                 ...           ...   
Todos los paises 9703        TOT           5.600905e+05  2.895279e+06   
                 9704        TOT           1.619459e+05  2.360655e+05   
                 9705        TOT           3.369923e+05  7.005600e+05   
                 9706        TOT           1.485024e+06  3.266286e+06   
                 9999        TOT           1.206252e+08  5.884525e+08   

                                                   2008          2009  \
ReporterISO3     ProductCode ReporterName                               
ABW              101         Aruba         1.285000e+00  4.476000e+00   
                 103         Aruba                  NaN  8.400000e-02   
                 104         Aruba         1.700000e-02           NaN   
                 105         Aruba         1.531000e+00  4.473000e+00   
                 106         Aruba         1.006900e+01  1.000700e+01   
...                                                 ...           ...   
Todos los paises 9703        TOT           3.143872e+06  2.971313e+06   
                 9704        TOT           2.376180e+05  2.069267e+05   
                 9705        TOT           9.198514e+05  6.852953e+05   
                 9706        TOT           3.093562e+06  1.908123e+06   
                 9999        TOT           7.235735e+08  6.607815e+08   

                                                   2017          2018  \
ReporterISO3     ProductCode ReporterName                               
ABW              101         Aruba         6.145000e+00  5.397000e+00   
                 103         Aruba                  NaN           NaN   
                 104         Aruba                  NaN           NaN   
                 105         Aruba         2.737400e+01  7.388400e+01   
                 106         Aruba         1.202400e+01  1.059400e+01   
...                                                 ...           ...   
Todos los paises 9703        TOT           5.316528e+06  5.450452e+06   
                 9704        TOT           1.719148e+05  1.864708e+05   
                 9705        TOT           1.596656e+06  1.682814e+06   
                 9706        TOT           3.154910e+06  3.511998e+06   
                 9999        TOT           8.181851e+08  9.128963e+08   

                                                   2019     prom_1719  \
ReporterISO3     ProductCode ReporterName                               
ABW              101         Aruba         8.483000e+00  6.680000e+00   
                 103         Aru

In [151]:
# armo subtotal por país
dfs = df_tot.sum(level=0)
dfs

,1997,1998,1999,2007,2008,2009,2017,2018,2019,prom_1719,prom_0709,prom_9799
ReporterISO3,,,,,,,,,,,,
ABW,0.000000e+00,0.000000e+00,0.000000e+00,9.789275e+04,1.008693e+05,1.357471e+05,8.907934e+04,6.976273e+04,8.362703e+04,8.252351e+04,1.282815e+05,0.000000e+00
AFG,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.400656e+05,4.034410e+05,0.000000e+00,8.755201e+05,0.000000e+00,8.755202e+05,5.056451e+05,0.000000e+00
AGO,0.000000e+00,0.000000e+00,0.000000e+00,4.417778e+07,0.000000e+00,4.063941e+07,3.490488e+07,4.209674e+07,0.000000e+00,3.898143e+07,4.240860e+07,0.000000e+00
ALB,1.376810e+05,2.076538e+05,3.510978e+05,1.077690e+06,5.440536e+05,1.087915e+06,2.302318e+06,2.875860e+06,0.000000e+00,2.629540e+06,9.163958e+05,2.410192e+05
AND,4.800039e+04,5.785890e+04,4.268060e+04,1.508462e+05,1.390993e+05,1.004779e+05,1.179656e+05,1.294482e+05,0.000000e+00,1.241620e+05,1.333405e+05,5.092940e+04
...,...,...,...,...,...,...,...,...,...,...,...,...
YEM,0.000000e+00,0.000000e+00,0.000000e+00,6.104268e+06,7.179989e+06,6.040706e+06,0.000000e+00,9.833120e+03,8.475465e+03,1.019419e+04,6.459510e+06,0.000000e+00
ZAF,2.248234e+07,1.958175e+07,2.106922e+07,6.402661e+07,7.396555e+07,5.386389e+07,8.794021e+07,9.192710e+07,8.779196e+07,8.921997e+07,6.397221e+07,2.513747e+07
ZMB,1.123429e+06,1.032257e+06,1.062903e+06,4.720011e+06,4.902756e+06,3.879827e+06,8.006790e+06,9.043157e+06,6.853336e+06,8.001964e+06,4.508956e+06,1.113924e+06


In [152]:
df_tot

1997          1998  \
ReporterISO3     ProductCode ReporterName                               
ABW              101         Aruba                  NaN           NaN   
                 103         Aruba                  NaN           NaN   
                 104         Aruba                  NaN           NaN   
                 105         Aruba                  NaN           NaN   
                 106         Aruba                  NaN           NaN   
...                                                 ...           ...   
Todos los paises 9703        TOT           4.995609e+05  5.288797e+05   
                 9704        TOT           2.039868e+05  1.748930e+05   
                 9705        TOT           2.854066e+05  3.425063e+05   
                 9706        TOT           1.445352e+06  1.337021e+06   
                 9999        TOT           1.199337e+08  1.107682e+08   

                                                   1999          2007  \
ReporterISO3     ProductCode ReporterName                               
ABW              101         Aruba                  NaN  1.809000e+00   
                 103         Aruba                  NaN           NaN   
                 104         Aruba                  NaN  6.420000e-01   
                 105         Aruba                  NaN  6.849000e+00   
                 106         Aruba                  NaN  5.834000e+00   
...                                                 ...           ...   
Todos los paises 9703        TOT           5.600905e+05  2.895279e+06   
                 9704        TOT           1.619459e+05  2.360655e+05   
                 9705        TOT           3.369923e+05  7.005600e+05   
                 9706        TOT           1.485024e+06  3.266286e+06   
                 9999        TOT           1.206252e+08  5.884525e+08   

                                                   2008          2009  \
ReporterISO3     ProductCode ReporterName                               
ABW              101         Aruba         1.285000e+00  4.476000e+00   
                 103         Aruba                  NaN  8.400000e-02   
                 104         Aruba         1.700000e-02           NaN   
                 105         Aruba         1.531000e+00  4.473000e+00   
                 106         Aruba         1.006900e+01  1.000700e+01   
...                                                 ...           ...   
Todos los paises 9703        TOT           3.143872e+06  2.971313e+06   
                 9704        TOT           2.376180e+05  2.069267e+05   
                 9705        TOT           9.198514e+05  6.852953e+05   
                 9706        TOT           3.093562e+06  1.908123e+06   
                 9999        TOT           7.235735e+08  6.607815e+08   

                                                   2017          2018  \
ReporterISO3     ProductCode ReporterName                               
ABW              101         Aruba         6.145000e+00  5.397000e+00   
                 103         Aruba                  NaN           NaN   
                 104         Aruba                  NaN           NaN   
                 105         Aruba         2.737400e+01  7.388400e+01   
                 106         Aruba         1.202400e+01  1.059400e+01   
...                                                 ...           ...   
Todos los paises 9703        TOT           5.316528e+06  5.450452e+06   
                 9704        TOT           1.719148e+05  1.864708e+05   
                 9705        TOT           1.596656e+06  1.682814e+06   
                 9706        TOT           3.154910e+06  3.511998e+06   
                 9999        TOT           8.181851e+08  9.128963e+08   

                                                   2019     prom_1719  \
ReporterISO3     ProductCode ReporterName                               
ABW              101         Aruba         8.483000e+00  6.680000e+00   
                 103         Aru

In [156]:
df_tot.append(dfs.assign([ProductCode=dfs.index.str[:-1] + '_Total', ReporterName=dfs.index.str[:-1]+ '_Total'])
                        .set_index(['ProductCode','ReporterName'], append=True))

SyntaxError: invalid syntax (<ipython-input-156-6cea82119f97>, line 1)